In [2]:
from model import get_final_df
import plotly.express as px
import pandas as pd
from collections import Counter


df = pd.read_parquet("../data/flats.parquet")
print(df.head(1))

  propertyCode  price        date propertyType   size  exterior  rooms  \
0     99170121  75000  2022-10-30       chalet  162.0     False      5   

             district  bathrooms  province  ... Zona_Monte_Real-Calicanto  \
0  Zona Casco Antiguo          3  València  ...                         0   

  Zona_Parc_Central-Hort_de_Trenor Zona_Poble_Nou  Zona_Pueblo  Zona_el_Molí  \
0                                0              0            0             0   

   Zona_pueblo nan  good  newdevelopment  renew  
0            0   0     0               0      1  

[1 rows x 173 columns]


In [3]:
df = df.loc[df["size"] < 300.0]

# df = df.loc[df["penthouse"] == True]

distr_list = [
    "Centro", "Ciutat_Vella", "LEixample", "Algirós", "Camins_al_Grau", "Quatre_Carreres", "Poblats_Marítims"]
df = df[df["district_renamed"].isin(distr_list)]

fig = px.scatter(x=df["size"], y=df["price"], color=df["district_renamed"], trendline="ols")
fig.show()

# Linear Regression

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df_linear = df[df["district_renamed"].isin(distr_list)]
df_linear = df_linear[distr_list + ["price", "rooms", 'size', 'exterior',  'bathrooms', 'priceChangeCount', 'good', 'newdevelopment', 'renew', 'chalet', 'countryHouse', 'duplex', 'flat', 'penthouse', 'studio', 'district_renamed']]

X = df_linear.drop(columns="price")
y = df_linear["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.drop(columns="district_renamed")
X_test = X_test.drop(columns="district_renamed")

print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

model = LinearRegression().fit(X_train, y_train)

4344
4344
1086
1086


In [5]:
r_sq = model.score(X_train, y_train)
print(f"coefficient of determination: {r_sq}")

print(f"intercept: {model.intercept_}")

print(f"coefficients: {model.coef_}")

coefficient of determination: 0.7399653867810836
intercept: 41027.605649341305
coefficients: [-191399.40782667  103370.05018476  123802.66756694  -15616.19893
    9143.564438    -17807.67573028  -11492.99970275  -19783.20809784
    2864.10196363   22679.27089944   51466.19621095    -508.24547319
 -131445.61884833  -88605.47153865 -166934.77943588  -54116.44134215
  -11735.62548024   41818.57497634  -18901.60458907   94087.57270689
  -51152.47627177]


In [7]:
y_pred = model.predict(X_test)
print(y_pred)

[271456.38625623 403025.93384689 208117.59026478 ... 266573.95203295
 194793.53642906 334337.66097237]


In [10]:
result_linear = X_test.copy()
result_linear["price_true"] = y_test
result_linear["price_predicted"] = y_pred

print(result_linear.head(2))

       Centro  Ciutat_Vella  LEixample  Algirós  Camins_al_Grau  \
36615       1             0          0        0               0   
34198       0             0          1        0               0   

       Quatre_Carreres  Poblats_Marítims  rooms   size  exterior  ...  \
36615                0                 0      3  215.0     False  ...   
34198                0                 0      4  132.0      True  ...   

       newdevelopment  renew  chalet  countryHouse  duplex  flat  penthouse  \
36615               0      0       1             0       0     0          0   
34198               0      1       0             0       0     1          0   

       studio  price_true  price_predicted  
36615       0      220000    271456.386256  
34198       0      495000    403025.933847  

[2 rows x 23 columns]
